## Entrenamiento de modelos supervisados para sentiments

Esta primera sección del notebook corresponde con el método de cargar df, se usa para entrenar los diferentes modelos implícitamente ya que siempre que se entrena un modelo de identificacion de sentimientos se usa el mismo df, por lo que es indispensable para el proceso de entrenamiento, el método lo que hace es cargar de los diferentes archivos .csv reunidos por todos los grupos los datos tageados, los normaliza y dependiendo de los parámetros de entrada del método también los lematizar y los balancea, además de normalizar las etiquetas de sentimientos como 0 para sentimientos positivos, 1 para sentimientos negativos y 2 para sentimientos neutrales, en la parte de lematización es importante resaltar que se usan lematizadores diferentes para ingles y español.

#### Carga de datos:
Para la carga de datos el método lee la carpeta de datos para sentimientos, donde se encuentran enumerados los archivos .csv, dependiendo del número de archivo el método trata al archivo de una manera u otra, ya que los rangos de números corresponden con los diferentes equipos, siendo los datos de 1 de aerolineas, 2, 3 y 4 de un equipo, 5, 6 y 7 de otro y 8 del ultimo, por esto se tratan de formas distintas, en esta primera parte se cargan solo las columnas de texto, idioma y el tag de sentimientos, por lo que además se deben renombrar algunas columnas en ciertos archivos, luego de esto se eliminan las filas que tengan valores nulos en cualquier columna, se eliminan las duplicados usando la columna texto com indicador y se filtran los datos cargados de tal forma que solo pasen los datos que tienen como idiomas ingles o español y como tag 0, 1 o 2, además se reinician los índices del df, posterior a esto usando expresiones regulares se eliminan los saltos de línea los espaciados de más de un espacio, los url, y los espacios al principio y fin de cada texto en el df, esto para normalizar los datos entregados.

#### Lematizacion:
La lematización, que es indicada por la segunda variable de entrada del método, la cual es binaria, se vale de la columna de idioma o lang para lematizar el texto usando un lematizador para ingles o español, para esto recorre todo el df identificando el idioma con la columna lang y lematizando el texto uno a uno según el lematizador correspondiente.

#### Balanceamiento de datos:
Por último para el balanceamiento de los datos se usa la primera variable del método, la cual también es binaria, para balancear el df de salida, para esto se crea una columna adicional en el df llamada sello, en sello se usan los atributos de idioma y de mourning en conjunto para crear una marca en el df según estos dos atributos, luego de completar los sellos se usan estos para que todos los sellos queden con la misma cantidad de datos en el df usando un muestreo aleatorio en df que fueron agrupados y separados del resto según su sello o marca, la cantidad de datos en cada muestra se determina usando la cantidad del df más pequeño resultante de la separación y agrupación por sellos, luego se unen de nuevo los df de muestras y se elimina la columna sellos, de esta forma la cantidad de datos de idioma y mourning se encuentran iguales en el df de salida en cada posible combinación.


In [1]:
# -----------------------------------------------------------------------------
# balance_data: 0 para retornar dataframe sin balancear o 1 para retornar
# dataframe balanceado.
# lematizacion: lematiza los textos usando nltk 0 para no lematizar y 1
# para retornar el texto lematizado.
# -----------------------------------------------------------------------------
def get_feelings_df(balance_data, lematizacion):
    from gensim.utils import any2unicode as unicode
    from nltk.stem import SnowballStemmer
    from sklearn.utils import resample
    import os, re, sys, pandas, unidecode

    feelings_folder = './entrenamiento de modelos/datos sentimientos'
    feelings_df = pandas.DataFrame(columns=['text', 'lang', 'sentiment'])
    path, subfolders, files_list = list(os.walk(feelings_folder))[0]
    files_list.sort()

    for i in range(len(files_list)):
        sys.stdout.write("\rPreparando df  " + str(round(((i + 1) / (len(files_list))) * 100, 2)) + "%")
        sys.stdout.flush()
        file_name, file_ext = files_list[i].split(".")

        if file_ext == 'csv':
            file_path = path + "/" + file_name + "." + file_ext
            df = pandas.read_csv(file_path, encoding='utf8', dtype=str, engine='python')
            numero_de_archivo = int(file_name.split("_")[0])

            if numero_de_archivo == 1:
                df = df.filter(['airline_sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 2 or numero_de_archivo == 3 or numero_de_archivo == 4:
                df = df.filter(['sentiment', 'text'])
                df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 5 or numero_de_archivo == 6 or numero_de_archivo == 7:
                df = df.filter(['polarity', 'text'])
                df.columns = ['sentiment', 'text']
                if numero_de_archivo in [5, 6]: df['lang'] = 'es'
                if numero_de_archivo == 7: df['lang'] = 'en'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

            if numero_de_archivo == 8:
                df = df.filter(['sentiment', 'text'])
                df.columns = ['sentiment', 'text']
                df['lang'] = 'es'
                df['sentiment'] = df.sentiment.map({'positive': '0', 'negative': '1', 'neutral': '2'})
                feelings_df = feelings_df.append(df)

    del df
    print("")
    feelings_df.dropna()
    feelings_df.drop_duplicates(subset=['text'], inplace=True)
    feelings_df = feelings_df.loc[feelings_df['sentiment'].isin(['0', '1', '2'])]
    feelings_df = feelings_df.loc[feelings_df['lang'].isin(['es', 'en'])]
    feelings_df.reset_index(drop=True, inplace=True)

    for i, row in feelings_df.iterrows():
        sys.stdout.write("\rNormalizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
        sys.stdout.flush()
        feelings_df.at[i, 'text'] = (
            re.sub('\s+', ' ',
                   re.sub(' +', ' ',
                          re.sub("http\S+", "",
                                 re.sub(r'\b(?=\w*[j])[aeiouj]{2,}\b', 'jajaja',
                                        re.sub(r'[\b@]\w+\s{1}', '', str(feelings_df.at[i, 'text'])
                                               )))))).strip()

    print("")

    if balance_data == 1:
        feelings_df.dropna()
        feelings_df["Sello"] = 0
        for i, row in feelings_df.iterrows():
            sys.stdout.write(
                "\rCreando sellos de balanceamiento " +
                str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2))
                + "%"
            )
            sys.stdout.flush()
            feelings_df.at[i, 'sello'] = str(feelings_df.at[i, 'lang']) + '_' + str(feelings_df.at[i, 'sentiment'])
        print("\nBalanceando df")
        min_len1 = int(min(feelings_df['sello'].value_counts()))
        df_0 = resample(feelings_df[feelings_df.sello == 'en_0'], replace=False, n_samples=min_len1, random_state=1)
        df_1 = resample(feelings_df[feelings_df.sello == 'en_1'], replace=False, n_samples=min_len1, random_state=1)
        df_2 = resample(feelings_df[feelings_df.sello == 'en_2'], replace=False, n_samples=min_len1, random_state=1)
        df_3 = resample(feelings_df[feelings_df.sello == 'es_0'], replace=False, n_samples=min_len1, random_state=1)
        df_4 = resample(feelings_df[feelings_df.sello == 'es_1'], replace=False, n_samples=min_len1, random_state=1)
        df_5 = resample(feelings_df[feelings_df.sello == 'es_2'], replace=False, n_samples=min_len1, random_state=1)
        feelings_df = pandas.concat([df_0, df_1, df_2, df_3, df_4, df_5])
        feelings_df = feelings_df.filter(['text', 'lang', 'sentiment'])
        feelings_df.reset_index(drop=True, inplace=True)

    if lematizacion == 1:
        stemmer_en = SnowballStemmer('english')
        stemmer_es = SnowballStemmer('spanish')
        for i, row in feelings_df.iterrows():
            sys.stdout.write("\rLematizando df " + str(round(((i + 1) / (feelings_df.shape[0])) * 100, 2)) + "%")
            sys.stdout.flush()
            if type(feelings_df.at[i, 'text']) is str and feelings_df.at[i, 'lang'] == 'es':
                feelings_df.at[i, 'text'] = stemmer_es.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
            elif type(feelings_df.at[i, 'text']) is str and feelings_df.at[i, 'lang'] == 'en':
                feelings_df.at[i, 'text'] = stemmer_en.stem(unidecode.unidecode(
                    unicode(feelings_df.at[i, 'text'].lower(), "utf-8"))
                )
        feelings_df.reset_index(drop=True, inplace=True)
        print("")

    print("Df sentiments entregado\n")
    feelings_df.reset_index(drop=True, inplace=True)
    return feelings_df


En esta segunda sección del notebook está el código necesario para entrenar los diferentes modelos supervisados de mourning dispuesto como método para ser implementado y usado fácilmente, este método se vale del de get_mourning_df, ya que este provee el df necesario para entrenar los diferentes modelos supervisados que se quieren entrenar.

El pipeline seguido para entrenar los modelos de mourning se basa primero en seleccionar el modelo que se va a entrenar, este se selecciona con el primer parámetro del método, el cual es una cadena donde se escriben las iniciales del modelo: 

GBT: para gradient boosting trees.
NN: para MPL o redes neuronales.
DT: para árboles de decisión.
RF: para random forest.
NB: para naive bayes.

Luego de seleccionar el modelo se carga el df haciendo uso del método get_mourning_df, los parámetros de este corresponden con los dos restantes del método de entrenamiento, luego de cargar el df este se divide según su idioma en 2 df, esto debido a que se entrenan modelos separados para ingles y español para integrar correctamente los lexicones posteriormente según el idioma, luego de esto se separan los df en los datos y etiquetas de prueba, este procedimiento también es independiente para cada idioma, posteriormente se vectorizan los datos tambien segun su idioma y se entrenan y prueban los modelos, en esta etapa se guardan los vocabularios de cada vectorizador aparte y también los modelos, esto con el fin de hacer uso de estos en otro programa posteriormente.

Por último se muestran los resultados del modelo entrenado.


In [2]:
def entrenar_modelos_sentiments_supervisados(modelo_entr, df_balanceado, df_lematizado):
    from nltk.corpus import stopwords
    from sklearn.naive_bayes import MultinomialNB
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.neural_network import MLPClassifier
    from sklearn.metrics import classification_report
    from sklearn.model_selection import train_test_split
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
    import pandas as pd
    import pickle, os, nltk

    modelos_es = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=6), n_estimators=4),
        'RF': RandomForestClassifier(n_estimators=14, max_depth=28),
        'NN': MLPClassifier(hidden_layer_sizes=(30, 1), max_iter=400),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': MultinomialNB()
    }

    modelos_en = {
        'GBT': AdaBoostClassifier(DecisionTreeClassifier(max_depth=6), n_estimators=4),
        'RF': RandomForestClassifier(n_estimators=14, max_depth=32),
        'NN': MLPClassifier(hidden_layer_sizes=(40, 1), max_iter=400),
        'DT': DecisionTreeClassifier(max_depth=16),
        'NB': MultinomialNB()
    }

    if modelo_entr in modelos_es and modelo_entr in modelos_en and 0 <= df_balanceado <= 1 and 0 <= df_lematizado <= 1:

        print("")
        nltk.download('stopwords')

        # ---------------- Asignacion de los modelos y vectorizadores.
        # -------- Español.
        modelo_es = modelos_es[modelo_entr]
        vectorizer_es = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('spanish'))
        # -------- Ingles.
        modelo_en = modelos_en[modelo_entr]
        vectorizer_en = TfidfVectorizer(use_idf=True, stop_words=stopwords.words('english'))

        # ---------------- Lectura y separacion de datos.
        df = pd.DataFrame(get_feelings_df(df_balanceado, df_lematizado))
        # -------- Español.
        df_es = df[df.lang == 'es']
        # -------- Ingles.
        df_en = df[df.lang == 'en']
        del df
        print("Separacion de datos por idioma terminada")

        # ---------------- Separacion en data y labels de entrenamiento.
        # -------- Español.
        data_train_es, data_test_es, label_train_es, label_test_es = train_test_split(
            df_es['text'], df_es['sentiment'], random_state=1
        )
        del df_es
        # -------- Ingles.
        data_train_en, data_test_en, label_train_en, label_test_en = train_test_split(
            df_en['text'], df_en['sentiment'], random_state=1
        )
        print("Division de datos terminada")
        del df_en

        # ---------------- vectorizacion de los textos.
        # -------- Español.
        training_data_es = vectorizer_es.fit_transform(data_train_es)
        testing_data_es = vectorizer_es.transform(data_test_es)
        del data_train_es, data_test_es
        print("Vectorizacion en español terminada")
        # -------- Ingles.
        training_data_en = vectorizer_en.fit_transform(data_train_en)
        testing_data_en = vectorizer_en.transform(data_test_en)
        del data_test_en, data_train_en
        print("Vectorizacion en ingles terminada")

        # ---------------- almacenamiento de los vocabularios.
        # -------- Español.
        ruta_vocabulario_es = "./entrenamiento de modelos/vocabularios/vocabulario_sentiment_es.pkl"
        if os.path.exists(ruta_vocabulario_es):
            os.remove(ruta_vocabulario_es)
        pickle.dump(vectorizer_es.vocabulary_, open(ruta_vocabulario_es, "wb"))
        print("Vocabulario para español almacenado en " + ruta_vocabulario_es)
        # -------- Ingles.
        ruta_vocabulario_en = "./entrenamiento de modelos/vocabularios/vocabulario_sentiment_en.pkl"
        if os.path.exists(ruta_vocabulario_en):
            os.remove(ruta_vocabulario_en)
        pickle.dump(vectorizer_en.vocabulary_, open(ruta_vocabulario_en, "wb"))
        print("Vocabulario para ingles almacenado en " + ruta_vocabulario_en)

        # ---------------- entrenamiento y guardado de los modelos.
        # -------- Español.
        ruta_modelo_es = './entrenamiento de modelos/modelos/' + str(type(modelo_es).__name__) + '_Sentiment_es.sav'
        modelo_es.fit(training_data_es, label_train_es)
        pickle.dump(modelo_es, open(ruta_modelo_es, 'wb'))
        print("Modelo de " + str(type(modelo_es).__name__) + " en español guardado en " + ruta_modelo_es)
        # -------- Ingles.
        ruta_modelo_en = './entrenamiento de modelos/modelos/' + str(type(modelo_en).__name__) + '_Sentiment_en.sav'
        modelo_en.fit(training_data_en, label_train_en)
        pickle.dump(modelo_en, open(ruta_modelo_en, 'wb'))
        print("Modelo de " + str(type(modelo_en).__name__) + " en ingles guardado en " + ruta_modelo_en)

        # ---------------- implementacion de los modelos.
        # -------- Español.
        predictions_es = modelo_es.predict(testing_data_es)
        # -------- Ingles.
        predictions_en = modelo_en.predict(testing_data_en)
        print("Predicciones terminadas")

        # ---------------- Resultados de los modelos.
        # -------- Español.
        print("\nResultados " + str(type(modelo_es).__name__) + " Español:\n")
        print(classification_report(label_test_es, predictions_es))
        # -------- Ingles.
        print("\nResultados " + str(type(modelo_en).__name__) + " Ingles:\n")
        print(classification_report(label_test_en, predictions_en))

    elif modelo_entr not in modelos_es or modelo_entr not in modelos_en or df_balanceado > 1 or df_balanceado < 0 or df_lematizado > 1 or df_lematizado < 0:

        print("Parametros incorrectos para entrenar modelo")
        

#### Entrenamiento de modelos NB:

In [3]:
entrenar_modelos_sentiments_supervisados('NB',1,1)

[nltk_data] Downloading package stopwords to /home/jose/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Preparando df  100.0%
Creando sellos de balanceamiento 100.0%
Balanceando df
Lematizando df 97.99%

#### Entrenamiento de modelos DT:

In [4]:
entrenar_modelos_sentiments_supervisados('DT',1,1)

Creando sellos de balanceamiento 23.14%

#### Entrenamiento de modelos RF:

In [ ]:
entrenar_modelos_sentiments_supervisados('RF',1,1)

#### Entrenamiento de modelos GBT:

In [ ]:
entrenar_modelos_sentiments_supervisados('GBT',1,1)

#### Entrenamiento de modelos NN:

In [ ]:
entrenar_modelos_sentiments_supervisados('NN',1,1)